In [ ]:
import os
from glob import glob
import logging
from pathlib import Path

from tqdm import tqdm
import h5py
import pickle

from harmonic_inference.data.piece import Piece, ScorePiece
import harmonic_inference.models.initial_chord_models as icm
from harmonic_inference.data.corpus_reading import load_clean_corpus_dfs
from harmonic_inference.models.joint_model import MODEL_CLASSES, HarmonicInferenceModel

logging.basicConfig(level=logging.DEBUG)

In [ ]:
files_df, measures_df, chords_df, notes_df = load_clean_corpus_dfs('corpus_data')

In [ ]:
files_df

In [ ]:
measures_df

In [ ]:
chords_df

In [ ]:
notes_df

In [ ]:
# Get model paths
model_paths = {}

for model in MODEL_CLASSES.keys():
    if model == "icm":
        continue

    model_paths[model] = os.path.join(
        "checkpoints", model, "lightning_logs", "version_*", "checkpoints", "*.ckpt"
    )

model_paths["icm"] = os.path.join("checkpoints", "icm", "initial_chord_prior.json")

In [ ]:
# Load models
models = {}
for model_name, model_class in MODEL_CLASSES.items():
    if model_name == "icm":
        continue

    possible_checkpoints = sorted(glob(model_paths[model_name]))
    if len(possible_checkpoints) == 0:
        logging.error(f"No checkpoints found for {model_name} in {model_paths[model_name]}")
        sys.exit(2)

    if len(possible_checkpoints) == 1:
        checkpoint = possible_checkpoints[0]
        logging.info(f"Loading checkpoint {checkpoint} for {model_name}.")

    else:
        checkpoint = possible_checkpoints[0]
        logging.info(f"Multiple checkpoints found for {model_name}. Loading {checkpoint}.")

    models[model_name] = model_class.load_from_checkpoint(checkpoint)
    models[model_name].freeze()

# Load icm json differently
logging.info(f"Loading checkpoint {model_paths['icm']} for icm.")
models["icm"] = icm.SimpleInitialChordModel(model_paths["icm"])

In [ ]:
h5_dir = Path("h5_data")
seed = 0

# Load validation data for ctm
h5_path = Path(h5_dir / f"ChordTransitionDataset_valid_seed_{seed}.h5")
with h5py.File(h5_path, "r") as h5_file:
    if "file_ids" not in h5_file:
        logging.error(f"file_ids not found in {h5_path}. Re-create with create_h5_data.py")
        sys.exit(1)

    file_ids = list(h5_file["file_ids"])

# Load from pkl if available
pkl_path = Path(h5_dir / f"pieces_valid_seed_{seed}.pkl")
if pkl_path.exists():
    with open(pkl_path, "rb") as pkl_file:
        piece_dicts = pickle.load(pkl_file)
    pieces = [
        ScorePiece(
            None,
            None,
            measures_df.loc[file_id],
            piece_dict=piece_dict,
            name=f"{file_id}: {files_df.loc[file_id, 'corpus_name']}/{files_df.loc[file_id, 'file_name']}",
        )
        for file_id, piece_dict in zip(file_ids, piece_dicts)
    ]

# Generate from dfs
else:
    pieces = []
    for file_id in tqdm(file_ids, desc="Loading Pieces"):
        pieces.append(
            ScorePiece(
                notes_df.loc[file_id],
                chords_df.loc[file_id],
                measures_df.loc[file_id],
                name=f"{file_id}: {files_df.loc[file_id, 'corpus_name']}/{files_df.loc[file_id, 'file_name']}",
            )
        )

In [ ]:
model = HarmonicInferenceModel(models)

state = model.get_harmony(pieces[0])
print(eu.evaluate(pieces[0], state))